### HAN's

Implementation based on paper here:
http://aclweb.org/anthology/N/N16/N16-1174.pdf
http://aclweb.org/anthology/D15-1167

Inspiration for code taken from here:
https://github.com/richliao/textClassifier/blob/master/textClassifierHATT.py

In [1]:
import sys, os, re, csv, codecs, gc, numpy as np, pandas as pd
import tensorflow as tf
#from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, Permute, GRU, Conv1D, LSTM, Embedding, Dropout, Activation, CuDNNLSTM, CuDNNGRU, concatenate, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D, GlobalAveragePooling1D, BatchNormalization, SpatialDropout1D, Dot
from keras.optimizers import Adam, RMSprop, Nadam
from keras.models import Model

from keras import initializers, regularizers, constraints, optimizers, layers
from keras_tqdm import TQDMNotebookCallback
import keras.backend as K
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import roc_auc_score
from functools import reduce
from keras.layers import Layer, PReLU, SpatialDropout1D, TimeDistributed
from keras import initializers
from sklearn.model_selection import cross_val_predict

from nltk.tokenize import word_tokenize, wordpunct_tokenize, TweetTokenizer, MWETokenizer, ToktokTokenizer, sent_tokenize
from nltk.corpus import stopwords

import unicodedata
from collections import Counter
import itertools

np.random.seed(786)

from SentenceTokenizer import SentenceTokenizer
from ZeroMaskedLayer import ZeroMaskedLayer
from AttentionLayer import AttentionLayer, AttentionWrapper, AttentionWithContext, Attention
from RocAucEvaluation import RocAucEvaluation
from HATTN_classifier import HATTNClassifier
from utils import outoffold_crossvalidator
from importlib import reload

/home/mohsin/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import HATTN_classifier
reload(HATTN_classifier)
from HATTN_classifier import HATTNClassifier

In [3]:
path = '../input/'
TRAIN_DATA_FILE=f'{path}train.csv'
TEST_DATA_FILE=f'{path}test.csv'

train = pd.read_csv(TRAIN_DATA_FILE)
test = pd.read_csv(TEST_DATA_FILE)

print(train.shape, test.shape)

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values

#Get validation folds
train['target_str'] = reduce(lambda x,y: x+y, [train[col].astype(str) for col in list_classes])
train['target_str'] = train['target_str'].replace('110101', '000000').replace('110110','000000')
cvlist1 = list(StratifiedKFold(n_splits=10, random_state=786).split(train, train['target_str'].astype('category')))
cvlist2 = list(StratifiedShuffleSplit(n_splits=5, test_size=0.05, random_state=786).split(train, train['target_str'].astype('category')))

(159571, 8) (153164, 2)


/home/mohsin/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


In [4]:
import pickle
with open("../utility/X_train_hattn_v1.pkl", "rb") as f:
    X_train = pickle.load(f)
with open("../utility/X_test_hattn_v1.pkl", "rb") as f:
    X_test = pickle.load(f) 
with open("../utility/embeddings_glove.pkl", "rb") as f:
    embedding_matrix = pickle.load(f)    

In [5]:
MAX_FEATURES = 200000
MAX_SENTENCE_LEN = 80
MAX_SENTENCES = 20

In [6]:
print(X_train.shape, X_test.shape)

(159571, 20, 80) (153164, 20, 80)


In [7]:
model = HATTNClassifier(MAX_SENTENCE_LEN, MAX_SENTENCES, MAX_FEATURES+1, 
                       word_embed_dim=300,
                       word_spatial_dropout=0.1,
                       sent_rnn_dim=300,
                       sent_rnn_type='gru',
                        sent_bidirectional_flag = True,
                        sent_rnn_layers=1,
                        doc_rnn_dim = 300,
                        doc_rnn_type='gru',
                        doc_bidirectional_flag=True,
                        doc_rnn_layers=1,
                        fc_layers=0,
                        batch_size=128,
                        epochs = 10,
                        optimizer='adam',
                        word_embed_kwargs={"weights":[embedding_matrix]}
                      )

In [ ]:
MODEL_IDENTIFIER="HATTN_base"

#Initialize model
def schedule(epoch):
    if epoch == 0:
        return 0.0015
    if epoch == 1:
        return 0.001
    if epoch == 2:
        return 0.001
    return 0.001    

callbacks=[LearningRateScheduler(schedule)]
    
check_filename="Model_"+str(MODEL_IDENTIFIER)+".check"
y_preds, y_trues, y_test = outoffold_crossvalidator(model, X_train, y, cvlist1, check_filename=check_filename,
                                                  predict_test=True, X_test=X_test, callbacks=callbacks, multiinput=False)

(?, 80, 300)
(?, 80, 600)
Epoch 1/10
143488/143593 [============================>.] - ETA: 0s - loss: 0.0509 - acc: 0.9815
 ROC-AUC - epoch: 1 - score: 0.987602 


Epoch 00001: val_score improved from -inf to 0.98760, saving model to Model_HATTN_base.check
143593/143593 [==============================] - 387s 3ms/step - loss: 0.0509 - acc: 0.9815
Epoch 2/10
143488/143593 [============================>.] - ETA: 0s - loss: 0.0375 - acc: 0.9851
 ROC-AUC - epoch: 2 - score: 0.989006 


Epoch 00002: val_score improved from 0.98760 to 0.98901, saving model to Model_HATTN_base.check
143593/143593 [==============================] - 389s 3ms/step - loss: 0.0375 - acc: 0.9851
Epoch 3/10
143488/143593 [============================>.] - ETA: 0s - loss: 0.0340 - acc: 0.9862
 ROC-AUC - epoch: 3 - score: 0.988511 


Epoch 00003: val_score did not improve
143593/143593 [==============================] - 387s 3ms/step - loss: 0.0340 - acc: 0.9862
Epoch 4/10
 27776/143593 [====>.........................

In [185]:
gc.collect()

433